In [16]:
import datasets as ds
import pandas as pd

human_eval = ds.load_dataset("openai/openai_humaneval")

print(human_eval)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


In [3]:
human_eval["test"].to_csv("human_eval.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

196170

In [11]:
# Rename all the functions in the dataset to "test"

import re

def lambda_rename (row: dict):
    fun_name: str = row["entry_point"]
    rename = "test"
    regex_pattern = rf"(def |>>> )({fun_name})(\()"
    replacement_str = rf"\1{rename}\3"

    regex_id = r"HumanEval"
    replacement_id = r"HumanEvalRenamed"

    renamed_id = re.sub(regex_id, replacement_id, row["task_id"])
    renamed_prompt = re.sub(regex_pattern, replacement_str, row["prompt"])
    row["task_id"] = renamed_id
    row["entry_point"] = rename
    row["prompt"] = renamed_prompt
    return row

renamed_set = human_eval["test"].map(lambda_rename)

human_eval["renamed"] = renamed_set

print(human_eval)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
    renamed: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


In [12]:
human_eval["renamed"].to_csv("human_eval_renamed.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

193303

In [31]:
import pandas as pd
import re


interference = pd.read_csv("data/nonsense.csv").sample(164)

In [36]:
def lambda_interfered (row: dict):
    serial = int(row["task_id"].split("/")[-1])


    regex_pattern = r'"""(?!.*""")'
    replace_str = f'Tip:\n\t{interference.values[serial][0]}\n"""'

    interfered_prompt = re.sub(regex_pattern, replace_str, row["prompt"], count=1, flags=re.DOTALL)
    row["prompt"] = interfered_prompt

    return row

# for i in range(len(human_eval["test"])):
human_eval["interfered"] = human_eval["test"].map(lambda_interfered)

print(human_eval)

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
    interfered: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})


In [37]:
human_eval["interfered"].to_csv("data/human_eval_interfered.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

206009